In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

Load in the data and examine

In [20]:
df = pd.read_csv('../../data/df_main.csv')

df.tail()

,year,Artist,from_country,to_country,points,total_points,rank,from_code2,from_code3,to_code2,...,Contains_Own_Language,gender,migration_v2p,population_p,prop_emigrants_v2p,migration_p2v,population_v,prop_emigrants_p2v,migration_pop_year,comps_since_last_win
21305,2022,Konstrakta,united kingdom,serbia,0,169,5.0,GB,GBR,RS,...,True,female,658.0,6899126.0,0.000095,9343.0,67081000.0,0.000139,2020.0,13
21306,2022,Chanel,united kingdom,spain,8,282,3.0,GB,GBR,ES,...,True,female,303193.0,47365655.0,0.006401,150892.0,67081000.0,0.002249,2020.0,51
21307,2022,Cornelia Jakobs,united kingdom,sweden,10,245,4.0,GB,GBR,SE,...,False,female,29715.0,10353442.0,0.002870,35824.0,67081000.0,0.000534,2020.0,5
21308,2022,Marius Bear,united kingdom,switzerland,0,28,18.0,GB,GBR,CH,...,False,male,45951.0,8638167.0,0.005320,23649.0,67081000.0,0.000353,2020.0,32
21309,2022,Kalush Orchestra,united kingdom,ukraine,7,379,1.0,GB,GBR,UA,...,True,group,NaN,NaN,NaN,22119.0,67081000.0,0.000330,2020.0,4


In [19]:
df['rank']

0         5.0
1        11.0
2        12.0
3        19.0
4        15.0
         ... 
21305     5.0
21306     3.0
21307     4.0
21308    18.0
21309     1.0
Name: rank, Length: 21310, dtype: float64

Lets determine the winners each year

In [11]:
# Get the total votes per to country per year
df['total_votes'] = df.groupby(['year', 'to_code2'])['points'].transform('sum')
df['max_votes'] = df.groupby(['year'])['total_votes'].transform('max')
df['max_votes_country'] = df['to_code2']
df.loc[df['total_votes'] != df['max_votes'], 'max_votes_country'] = np.nan

winners = df[['max_votes_country', 'year']]
winners = winners.loc[~winners['max_votes_country'].isna()]
winners = winners.drop_duplicates()
winners = winners.set_index('year')

The countries who have won multiple times in this time frame are:

In [12]:
winners['max_votes_country'].value_counts().loc[winners['max_votes_country'].value_counts() > 1]

UA    3
SE    3
IL    2
DK    2
Name: max_votes_country, dtype: int64

,year,Artist,from_country,to_country,points,from_code2,from_code3,to_code2,to_code3,Official_languages,...,population_p,prop_emigrants_v2p,migration_p2v,population_v,prop_emigrants_p2v,migration_pop_year,comps_since_last_win,total_votes,max_votes,max_votes_country
0,1998,Danijela,belgium,croatia,5,BE,BEL,HR,HRV,croatian,...,4620030.0,0.000044,72.0,10136811.0,0.000007,1995.0,42,131,172,NaN
1,1998,Michael Hajiyanni,belgium,cyprus,2,BE,BEL,CY,CYP,greek turkish,...,862418.0,0.000107,77.0,10136811.0,0.000008,1995.0,42,37,172,NaN
2,1998,Koit Toome,belgium,estonia,0,BE,BEL,EE,EST,võro estonian,...,1436634.0,0.000000,57.0,10136811.0,0.000006,1995.0,42,36,172,NaN
3,1998,Vlado Janevski,belgium,north macedonia,0,BE,BEL,MK,MKD,macedonian albanian,...,NaN,NaN,120.0,10136811.0,0.000012,1995.0,42,16,172,NaN
4,1998,Edea,belgium,finland,0,BE,BEL,FI,FIN,finnish swedish,...,5107790.0,0.000028,1541.0,10136811.0,0.000152,1995.0,42,22,172,NaN


In [ ]:
my_cmap = plt.get_cmap("magma_r")
rescale = lambda y: (y) / (26)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6), sharey=True, sharex=True, squeeze=True)

country = "UA"
ax=ax1
df_plot=df

#def plot_country(country, ax, df_plot):
df_temp = df_plot.loc[df_plot['to_country'] == country]

# add dummy data for years not included in the data
# missing_years = list(set(range(1997, 2023)) - set(df_temp['Year']))

# sort by year
df_temp = df_temp.sort_values('year', ascending=False)    
df_temp['Position'] = df_temp['Position'].astype(float)

# matplotlib bar chart
ax.bar(df_temp['Year'], df_temp['Votes'], 
        color=my_cmap(rescale(df_temp['Position'])))

ax.scatter(2020, 10, s=25, color='grey', marker="_")

# annotate if nan (did not perform)
for i, v in enumerate(df_temp['Votes']):
    if np.isnan(v):
        ax.scatter(df_temp['Year'].iloc[i], 10, s=25, color='grey', marker="x")


winner = df_temp.loc[df_temp['Position'] == 1]
if winner.shape[0] > 0:
    ax.scatter(winner['Year'].iloc[0], winner['Votes'].iloc[0], s=50, 
                color='gold', marker="*", edgecolor='black', zorder=5)



# look up the country name from the country code dict value
country_name = list(country_codes_dict.keys())[list(country_codes_dict.values()).index(country)]

ax.set_xlim(1997, 2023)
ax.set_title(country_name)
